In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

2024-12-15 00:23:20.007434: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-15 00:23:20.264760: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-15 00:23:20.265876: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 00:23:22.197627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_data(start, end):
    symbol = 'BTC-USD' 
    data = yf.download(symbol, start=start, end=end)
    data.reset_index(inplace=True)
    data['Date'] = data['Date'].dt.date
    return data

In [3]:
start_date = '2020-01-01'
end_date = '2024-01-01'
bitcoin_data = get_data(start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(bitcoin_data[['Adj Close']]) 

In [5]:
def create_sequences(data, time_step=60):
    X = []
    y = []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step, 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [6]:
time_step = 60
X, y = create_sequences(scaled_data, time_step)

X = X.reshape(X.shape[0], X.shape[1], 1)

In [7]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(units=50, return_sequences=False),
    tf.keras.layers.Dense(units=1)
])

In [9]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [10]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
35/35 [==============================] - 6s 30ms/step - loss: 0.0200
Epoch 2/10
35/35 [==============================] - 1s 30ms/step - loss: 0.0026
Epoch 3/10
35/35 [==============================] - 1s 31ms/step - loss: 0.0022
Epoch 4/10
35/35 [==============================] - 1s 31ms/step - loss: 0.0021
Epoch 5/10
35/35 [==============================] - 1s 30ms/step - loss: 0.0020
Epoch 6/10
35/35 [==============================] - 1s 31ms/step - loss: 0.0018
Epoch 7/10
35/35 [==============================] - 1s 30ms/step - loss: 0.0018
Epoch 8/10
35/35 [==============================] - 1s 30ms/step - loss: 0.0016
Epoch 9/10
35/35 [==============================] - 1s 30ms/step - loss: 0.0018
Epoch 10/10
35/35 [==============================] - 1s 30ms/step - loss: 0.0015


In [11]:
import joblib 
joblib.dump(model, 'model.pkl')

['model.pkl']